In [0]:
import os
import sys
import numpy as np
import cv2

In [ ]:
st_name='fg_'
path='fg/'

In [ ]:
# Rename background Images name
for count, filename in enumerate(os.listdir(path)):
    if filename.endswith('.jpg'):
        i=str(count+1).zfill(3)
        os.rename(os.path.join(path,filename), os.path.join(path,st_name+i+'.png'))

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
path='/content/drive/My Drive/S15.zip'
!unzip '{path}' -d '/content/Dataset'

Archive:  /content/drive/My Drive/S15.zip
   creating: /content/Dataset/bg/
  inflating: /content/Dataset/bg/1572522895php7OCj0a.jpeg  
 extracting: /content/Dataset/bg/194015-d19a0f43-e1a0-4142-8dfe-5c08c1ee57f5.png  
 extracting: /content/Dataset/bg/7272006_101922_00.png  
  inflating: /content/Dataset/bg/AF2508F9-79F2-44D9-B566-6D1C7CEE8C88_1_105_c.jpeg  
  inflating: /content/Dataset/bg/bg_001.jpg  
  inflating: /content/Dataset/bg/bg_002.jpg  
  inflating: /content/Dataset/bg/bg_003.jpg  
  inflating: /content/Dataset/bg/bg_004.jpg  
 extracting: /content/Dataset/bg/bg_005.jpg  
 extracting: /content/Dataset/bg/bg_007.jpg  
  inflating: /content/Dataset/bg/bg_009.jpg  
  inflating: /content/Dataset/bg/bg_010.jpg  
  inflating: /content/Dataset/bg/bg_011.jpg  
  inflating: /content/Dataset/bg/bg_012.jpg  
  inflating: /content/Dataset/bg/bg_013.jpg  
  inflating: /content/Dataset/bg/bg_014.jpg  
  inflating: /content/Dataset/bg/bg_015.jpg  
  inflating: /content/Dataset/bg/bg_016.j

In [0]:
import os
os.chdir('/content/Dataset/')

In [0]:
pathfgbg='fg_bg'
os.mkdir(pathfgbg)
pathmask='fg_bg_mask'
os.mkdir(pathmask)

In [0]:
def creatingimgdataset(bg_image, fg_image):
    nx=np.random.randint(len(bg_image[1])-len(fg_image[1]))
    ny=np.random.randint(len(bg_image[0])-len(fg_image[0]))
    fgrows, fgcols, fgchannels = fg_image.shape
    final_image=bg_image.copy()
    bgIntersection=bg_image[nx:nx+fgrows, ny:ny+fgcols].copy()
    # creating mask for fg image by converting to grayscale
    fg_image_gray=cv2.cvtColor(fg_image, cv2.COLOR_BGR2GRAY)
    _,fg_image_binary=cv2.threshold(fg_image_gray, 1, 255, cv2.THRESH_BINARY) #preserving maximum information by loewring thresh value to 1. 
    #Converting to Binary Image from grayscale image
    fg_image_binary_invert=cv2.bitwise_not(fg_image_binary)
    # Now blackout the area of intersection of bg_Image.
    # The image size may be have varying length. True for transparent images also.
    # only blackout those areas where my logos lies. The mask will enable the operation
    # where it doesnot zero. That's why with invert mask. For others it return zeros
    bg_image_selection=cv2.bitwise_and(bgIntersection,bgIntersection, mask=fg_image_binary_invert)
    fg_image_selection=cv2.bitwise_and(fg_image, fg_image, mask=fg_image_binary)
    # Put a foreground image on background image
    # adding both images for the section
    final_image_selection = cv2.add(bg_image_selection, fg_image_selection)
    final_image[nx:nx+fgrows, ny:ny+fgcols]=final_image_selection
    # mask of the image
    bg_image_mask=np.full((bg_image.shape[0], bg_image.shape[1]), 0, dtype=np.uint8)
    bg_image_mask[nx:nx+fgrows, ny:ny+fgcols]=fg_image_binary
    return bg_image_mask, final_image
    #cv2.imshow("Image",bg_image_mask)
    #cv2.waitKey(0)
    #cv2.destroyAllWindows()

In [9]:
pathbg='bg/'
pathfg='fg_final/'
pathfgbg='fg_bg/'
pathmask='fg_bg_mask/'
pathbgfinal='bg_final/'
os.mkdir(pathbgfinal)
scaleFactor=0.99
filenames=[]
for countbg, filenamebg in enumerate(os.listdir(pathbg)):
    maskpath=os.path.join(pathmask,str(countbg+1).zfill(3))
    fgbgpath=os.path.join(pathfgbg,str(countbg+1).zfill(3))
    bgfinalpath=os.path.join(pathbgfinal,str(countbg+1).zfill(3))
    if not(filenamebg.endswith('.jpg')):
        continue
    bg_image=cv2.imread(os.path.join(pathbg, filenamebg))
    bgx, bgy = len(bg_image[1]),len(bg_image[0])
    while bgx > 224 or bgy > 224: bgx,bgy = bgx*scaleFactor,bgy*scaleFactor
    resized_bg_image = cv2.resize(bg_image,(int(bgx),int(bgy)),interpolation = cv2.INTER_AREA)
    for countfg, filenamefg in enumerate(os.listdir(pathfg)):
        if countfg%10==0:
            maskpath1=os.path.join(maskpath,str(countfg+1).zfill(3))
            fgbgpath1=os.path.join(fgbgpath,str(countfg+1).zfill(3))
            os.makedirs(maskpath1)
            os.makedirs(fgbgpath1)
            count=1
        if not(filenamefg.endswith('.png')):
            continue
        fg_image=cv2.imread(os.path.join(pathfg, filenamefg))
        fgx, fgy = len(fg_image[1]),len(fg_image[0])
        while fgx > 100 or fgy > 100: fgx,fgy = fgx*scaleFactor,fgy*scaleFactor
        resized_fg_image = cv2.resize(fg_image,(int(fgx),int(fgy)),interpolation = cv2.INTER_AREA)
        # creating dataset for flips and Nonflips
        for counter in range(1,41):
            if counter==21:resized_fg_image=cv2.flip(resized_fg_image, 1)
            fg_bg_mask, fg_bg_image=creatingimgdataset(resized_bg_image, resized_fg_image)
            cv2.imwrite(os.path.join(maskpath1,str(count).zfill(3)+'.jpg'),fg_bg_mask)
            cv2.imwrite(os.path.join(fgbgpath1,str(count).zfill(3)+'.jpg'),fg_bg_image)
            count+=1
    print(filenamebg)
    cv2.imwrite(bgfinalpath+'.jpg',resized_bg_image)
    filenames.append(filenamebg)
    if len(filenames)==100:break;

bg_164.jpg
bg_064.jpg
bg_040.jpg
bg_079.jpg
bg_058.jpg
bg_038.jpg
bg_085.jpg
bg_048.jpg
bg_073.jpg
bg_105.jpg
bg_092.jpg
bg_005.jpg
bg_157.jpg
bg_035.jpg
bg_086.jpg
bg_146.jpg
bg_169.jpg
bg_089.jpg
bg_052.jpg
bg_129.jpg
bg_059.jpg
bg_116.jpg
bg_127.jpg
bg_007.jpg
bg_070.jpg
bg_063.jpg
bg_056.jpg
bg_153.jpg
bg_027.jpg
bg_050.jpg
bg_143.jpg
bg_112.jpg
bg_115.jpg
bg_155.jpg
bg_013.jpg
bg_125.jpg
bg_137.jpg
bg_104.jpg
bg_174.jpg
bg_114.jpg
bg_066.jpg
bg_119.jpg
bg_024.jpg
bg_111.jpg
bg_011.jpg
bg_087.jpg
bg_033.jpg
bg_062.jpg
bg_043.jpg
bg_018.jpg
bg_002.jpg
bg_099.jpg
bg_076.jpg
bg_069.jpg
bg_017.jpg
bg_159.jpg
bg_075.jpg
bg_047.jpg
bg_049.jpg
bg_162.jpg
bg_051.jpg
bg_124.jpg
bg_080.jpg
bg_074.jpg
bg_101.jpg
bg_078.jpg
bg_140.jpg
bg_154.jpg
bg_166.jpg
bg_126.jpg
bg_161.jpg
bg_106.jpg
bg_029.jpg
bg_081.jpg
bg_003.jpg
bg_170.jpg
bg_046.jpg
bg_042.jpg
bg_110.jpg
bg_057.jpg
bg_142.jpg
bg_041.jpg
bg_077.jpg
bg_071.jpg
bg_108.jpg
bg_121.jpg
bg_160.jpg
bg_036.jpg
bg_163.jpg
bg_149.jpg
bg_004.jpg

In [10]:
!tar -cvf '/content/Dataset.tar' '/content/Dataset/'

Streaming output truncated to the last 5000 lines.
/content/Dataset/fg_bg/045/051/395.jpg
/content/Dataset/fg_bg/045/051/283.jpg
/content/Dataset/fg_bg/045/051/227.jpg
/content/Dataset/fg_bg/045/051/137.jpg
/content/Dataset/fg_bg/045/051/025.jpg
/content/Dataset/fg_bg/045/051/247.jpg
/content/Dataset/fg_bg/045/051/326.jpg
/content/Dataset/fg_bg/045/051/180.jpg
/content/Dataset/fg_bg/045/051/318.jpg
/content/Dataset/fg_bg/045/051/154.jpg
/content/Dataset/fg_bg/045/051/157.jpg
/content/Dataset/fg_bg/045/021/
/content/Dataset/fg_bg/045/021/116.jpg
/content/Dataset/fg_bg/045/021/034.jpg
/content/Dataset/fg_bg/045/021/121.jpg
/content/Dataset/fg_bg/045/021/194.jpg
/content/Dataset/fg_bg/045/021/273.jpg
/content/Dataset/fg_bg/045/021/056.jpg
/content/Dataset/fg_bg/045/021/079.jpg
/content/Dataset/fg_bg/045/021/208.jpg
/content/Dataset/fg_bg/045/021/218.jpg
/content/Dataset/fg_bg/045/021/356.jpg
/content/Dataset/fg_bg/045/021/135.jpg
/content/Dataset/fg_bg/045/021/377.jpg
/content/Dataset/fg_

In [12]:
!7z a '/content/bg.7z' '/content/Dataset/bg_final/*'


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.30GHz (306F0),ASM,AES-NI)

Scanning the drive:
  0M Scan  /content/Dataset/bg_final/                                     100 files, 2341905 bytes (2288 KiB)

Creating archive: /content/bg.7z

Items to compress: 100

  0%     19% 24 + 026.jpg                  40% 43 + 046.jpg                  55% 59 + 063.jpg                  72% 79 + 083.jpg                  92% 97 + 103.jpg                 
Files read from disk: 100
Archive size: 2286969 bytes (2234 KiB)
Everything is Ok


In [13]:
!7z a '/content/fgbg.7z' '/content/Dataset/fg_bg/*'


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.30GHz (306F0),ASM,AES-NI)

Scanning the drive:
  0M Scan  /content/Dataset/fg_bg/                                  377M 15644 Scan  /content/Dataset/fg_bg/084/091/                                                697M 30084 Scan  /content/Dataset/fg_bg/061/071/                                                1148M 48133 Scan  /content/Dataset/fg_bg/049/                                             1574M 66183 Scan  /content/Dataset/fg_bg/100/071/

In [14]:
!7z a '/content/fgbgmask.7z' '/content/Dataset/fg_bg_mask/*'


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.30GHz (306F0),ASM,AES-NI)

Scanning the drive:
  0M Scan  /content/Dataset/fg_bg_mask/                                        55M 19655 Scan  /content/Dataset/fg_bg_mask/042/091/                                                     109M 39309 Scan  /content/Dataset/fg_bg_mask/006/021/                                                     162M 58161 Scan  /content/Dataset/fg_bg_mask/102/071/                                                     

In [11]:
%cp -av '/content/Dataset.tar' '/content/drive/My Drive/'

'/content/Dataset.tar' -> '/content/drive/My Drive/Dataset.tar'
